In [1]:
import numpy as np

class calculator:
    def __init__(self, in_mat_sz):
        self.in_mat_sz = in_mat_sz
        self.conv_cnt = []
        self.maxpool_cnt = []
        self.nn_cnt = []
        self.dropout_nn_cnt = []
        self.weights = []
        self.biases = []
        self.params = []
    
    def calc_z(self, n, m, s, p):
        return (n - m + 2 * p) / s + 1
    
    def get_conv_cnt(self, kern_sz, s, p):
        print('-'*10)
        print(f'input matrix size : {self.in_mat_sz}')
        print(f'kernel size : {kern_sz}')
        print(f'stride : {s}')
        print(f'padding : {p}')
        n = self.in_mat_sz[0]
        k1 = self.in_mat_sz[-1]
        
        m = kern_sz[0]
        k2 = kern_sz[-1]
        
        z = self.calc_z(n, m, s, p)
        
        # (m^2 + c) * z^2 * k + z(biases)
        conv_cnt = (m**2 + k1) * z**2 * k2 + z
        relu_cnt = conv_cnt + z**2
        
        self.in_mat_sz = [z, z, k2]
        self.conv_cnt.append(int(relu_cnt))
        self.weights.append(int(m**2 * k1 * k2))
        self.biases.append(int(k2))
        self.params.append(int(self.weights[-1] + self.biases[-1]))
        
        print('-'*10)
        print(f'output matrix size : {self.in_mat_sz}')
        print(f'# of opeartion : {self.conv_cnt[-1]:,}')
        print(f'# of weight : {self.weights[-1]:,}')
        print(f'# of bias : {self.biases[-1]:,}')
        print(f'# of param : {self.params[-1]:,}')
        print('-'*10)
    
    def get_maxpool_cnt(self, kern_sz, s):
        print('-'*10)
        print(f'input matrix size : {self.in_mat_sz}')
        print(f'kernel size : {kern_sz}')
        print(f'stride : {s}')
        n = self.in_mat_sz[0]
        k = self.in_mat_sz[-1]
        
        m = kern_sz[0]
        
        z = self.calc_z(n, m, s, 0)
        
        maxpool_cnt = k * m**2 * z**2
        
        self.in_mat_sz = [z, z, k]
        self.maxpool_cnt.append(int(maxpool_cnt))
        
        print('-'*10)
        print(f'output matrix size : {self.in_mat_sz}')
        print(f'# of opeartion : {self.maxpool_cnt[-1]:,}')
        print('-'*10)
    
    def get_nn_cnt(self, net_sz):
        print('-'*10)
        print(f'input network size : {self.in_mat_sz}')
        print(f'network size : {net_sz}')
        in_net_sz = np.prod(self.in_mat_sz)
        to_net_sz = np.prod(net_sz)
        
        # TODO: calculate the number of operation
        
        nn_cnt = (2 * in_net_sz + 1) * to_net_sz
        dropout_nn_cnt = (2 * in_net_sz + 1) * to_net_sz * 0.5
        
        self.nn_cnt.append(int(nn_cnt))
        self.dropout_nn_cnt.append(int(dropout_nn_cnt))
        self.weights.append(int(in_net_sz * to_net_sz))
        self.biases.append(int(to_net_sz))
        self.params.append(int(self.weights[-1] + self.biases[-1]))
        
        self.in_mat_sz = net_sz
        
        print('-'*10)
        print(f'output network size : {self.in_mat_sz}')
        print(f'# of operation w/o dropout : {self.nn_cnt[-1]:,}')
        print(f'# of operation w/ dropout : {self.dropout_nn_cnt[-1]:,}')
        print(f'# of weight : {self.weights[-1]:,}')
        print(f'# of bias : {self.biases[-1]:,}')
        print(f'# of param : {self.params[-1]:,}')
        print('-'*10)
    
    def cnt_sum(self, dropout=0):
        return sum(self.conv_cnt) + sum(self.maxpool_cnt) + (sum(self.nn_cnt) if dropout == 0 else sum(self.dropout_nn_cnt))

In [2]:
AlexNet = calculator([227, 227, 3])

In [3]:
AlexNet.get_conv_cnt([11, 11, 96], 4, 0)

----------
input matrix size : [227, 227, 3]
kernel size : [11, 11, 96]
stride : 4
padding : 0
----------
output matrix size : [55.0, 55.0, 96]
# of opeartion : 36,012,680
# of weight : 34,848
# of bias : 96
# of param : 34,944
----------


In [4]:
AlexNet.get_maxpool_cnt([3, 3], 2)

----------
input matrix size : [55.0, 55.0, 96]
kernel size : [3, 3]
stride : 2
----------
output matrix size : [27.0, 27.0, 96]
# of opeartion : 629,856
----------


In [5]:
AlexNet.get_conv_cnt([5, 5, 256], 1, 2)

----------
input matrix size : [27.0, 27.0, 96]
kernel size : [5, 5, 256]
stride : 1
padding : 2
----------
output matrix size : [27.0, 27.0, 256]
# of opeartion : 22,582,260
# of weight : 614,400
# of bias : 256
# of param : 614,656
----------


In [6]:
AlexNet.get_maxpool_cnt([3, 3], 2)

----------
input matrix size : [27.0, 27.0, 256]
kernel size : [3, 3]
stride : 2
----------
output matrix size : [13.0, 13.0, 256]
# of opeartion : 389,376
----------


In [7]:
AlexNet.get_conv_cnt([3, 3, 384], 1, 1)

----------
input matrix size : [13.0, 13.0, 256]
kernel size : [3, 3, 384]
stride : 1
padding : 1
----------
output matrix size : [13.0, 13.0, 384]
# of opeartion : 17,197,622
# of weight : 884,736
# of bias : 384
# of param : 885,120
----------


In [8]:
AlexNet.get_conv_cnt([3, 3, 384], 1, 1)

----------
input matrix size : [13.0, 13.0, 384]
kernel size : [3, 3, 384]
stride : 1
padding : 1
----------
output matrix size : [13.0, 13.0, 384]
# of opeartion : 25,504,310
# of weight : 1,327,104
# of bias : 384
# of param : 1,327,488
----------


In [9]:
AlexNet.get_conv_cnt([3, 3, 256], 1, 1)

----------
input matrix size : [13.0, 13.0, 384]
kernel size : [3, 3, 256]
stride : 1
padding : 1
----------
output matrix size : [13.0, 13.0, 256]
# of opeartion : 17,002,934
# of weight : 884,736
# of bias : 256
# of param : 884,992
----------


In [10]:
AlexNet.get_maxpool_cnt([3, 3], 2)

----------
input matrix size : [13.0, 13.0, 256]
kernel size : [3, 3]
stride : 2
----------
output matrix size : [6.0, 6.0, 256]
# of opeartion : 82,944
----------


In [11]:
AlexNet.get_nn_cnt([1, 4096])

----------
input network size : [6.0, 6.0, 256]
network size : [1, 4096]
----------
output network size : [1, 4096]
# of operation w/o dropout : 75,501,568
# of operation w/ dropout : 37,750,784
# of weight : 37,748,736
# of bias : 4,096
# of param : 37,752,832
----------


In [12]:
AlexNet.get_nn_cnt([1, 4096])

----------
input network size : [1, 4096]
network size : [1, 4096]
----------
output network size : [1, 4096]
# of operation w/o dropout : 33,558,528
# of operation w/ dropout : 16,779,264
# of weight : 16,777,216
# of bias : 4,096
# of param : 16,781,312
----------


In [13]:
AlexNet.get_nn_cnt([1, 1000])

----------
input network size : [1, 4096]
network size : [1, 1000]
----------
output network size : [1, 1000]
# of operation w/o dropout : 8,193,000
# of operation w/ dropout : 4,096,500
# of weight : 4,096,000
# of bias : 1,000
# of param : 4,097,000
----------


In [14]:
print(f'{AlexNet.cnt_sum():,}')

236,655,078


In [15]:
print(f'{AlexNet.cnt_sum(1):,}')

178,028,530


In [16]:
print(AlexNet.conv_cnt)

[36012680, 22582260, 17197622, 25504310, 17002934]


In [17]:
print(AlexNet.maxpool_cnt)

[629856, 389376, 82944]


In [18]:
print(AlexNet.weights)

[34848, 614400, 884736, 1327104, 884736, 37748736, 16777216, 4096000]


In [19]:
print(AlexNet.biases)

[96, 256, 384, 384, 256, 4096, 4096, 1000]


In [20]:
print(AlexNet.params)

[34944, 614656, 885120, 1327488, 884992, 37752832, 16781312, 4097000]


In [21]:
print(AlexNet.nn_cnt)

[75501568, 33558528, 8193000]


In [22]:
print(AlexNet.dropout_nn_cnt)

[37750784, 16779264, 4096500]
